In [1]:
import pandas as pd

In [2]:
from sklearn.model_selection import train_test_split
from cnn import ChessDataset, ChessEvaluationConvolutionalNetwork, ConvolutionInputModel
from torch.utils.data import DataLoader
import torch


df = pd.read_csv('../../data/chess_data.csv')

df['board_vec'] = df.apply(lambda row: ConvolutionInputModel.fen_to_feature_array(row['FEN'])[0].squeeze(0), axis=1)
df['meta_vec'] = df.apply(lambda row: ConvolutionInputModel.fen_to_feature_array(row['FEN'])[1].squeeze(0), axis=1)

# Debug print for checking output shape of board_vec and meta_vec
print(df['board_vec'].iloc[0].shape)  # Print the shape of the first board_vec
print(df['meta_vec'].iloc[0].shape)   # Print the shape of the first meta_vec

X = df[['board_vec', 'meta_vec']]
df['Evaluation'] = pd.to_numeric(df['Evaluation'], errors='coerce')
Y = df['Evaluation']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)  # Convert to a column tensor
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

train_dataset = ChessDataset(X_train, y_train_tensor)
test_dataset = ChessDataset(X_test, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)



torch.Size([12, 8, 8])
torch.Size([5])


In [3]:
import torch.nn as nn
import torch.optim as optim

batch_size = 32
epochs = 10
lr = 0.001


model = ChessEvaluationConvolutionalNetwork()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# Training Loop
for epoch in range(epochs):
    total_loss = 0.0
    model.train()

    for x_board, x_meta, target in train_loader:
        print(x_board.shape, "board shape")
        print(x_meta.shape, "metadata shape")
        print(target.shape, "Target Shape")
        optimizer.zero_grad()
        output = model(x_board, x_meta)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * x_board.size(0)

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

KeyError: 48278